In [ ]:
import json
import random
import commentjson
import re

my_label = 0
templatePath = 'intentTemplate/title_auther_ner_template.jsonc'
entityPath = 'entities/titles.jsonc'
authorPath = 'entities/authors.jsonc'
entityMain = 'genre'
entityAuthor = 'AUTHOR'
# gernePath = 'entities/genres.jsonc' # 원본에 있었던 변수, 필요시 주석 해제

def choose_josa(word, josa_with_batchim, josa_without_batchim):
    if not word:
        return ""
    last_char = word[-1]
    if '가' <= last_char <= '힣':
        has_batchim = (ord(last_char) - ord('가')) % 28 > 0
        return josa_with_batchim if has_batchim else josa_without_batchim
    return josa_without_batchim

def process_korean_template(template, title, author=""):
    text = template.replace('{}', title)
    text = text.replace('[]', author)
    text = text.replace('{이}', choose_josa(title, '이', ''))
    text = text.replace('{은}', choose_josa(title, '은', '는'))
    text = text.replace('{를}', choose_josa(title, '을', '를'))
    text = text.replace('{과}', choose_josa(title, '과', '와'))
    text = text.replace('{가}', choose_josa(title, '이', '가'))
    text = text.replace('{으로}', choose_josa(title, '으로', '로'))
    text = text.replace('[이]', choose_josa(author, '이', ''))
    text = text.replace('[은]', choose_josa(author, '은', '는'))
    text = text.replace('[를]', choose_josa(author, '을', '를'))
    text = text.replace('[과]', choose_josa(author, '과', '와'))
    text = text.replace('[가]', choose_josa(author, '이', '가'))
    text = text.replace('[으로]', choose_josa(author, '으로', '로'))
    return text

def generate_ner_data(templates, titles, authors, current_intent_label, main_entity_type, author_entity_type):
    output_data = []
    format_keywords = ["책", "도서", "자료"]
    exclude_words = ["도서관", "자료실", "자료집", "자료형"]

    for template in templates:
        title = random.choice(titles) if titles else ""
        author = random.choice(authors) if authors else ""
        formatted_text = process_korean_template(template, title, author)

        current_entities = []

        if title:
            title_start = formatted_text.find(title)
            if title_start != -1:
                title_end = title_start + len(title)
                current_entities.append({
                    "start": title_start,
                    "end": title_end,
                    "type": main_entity_type
                })

        if author:
            author_start = -1
            search_offset = 0
            while True: # 저자명과 제목명이 겹치지 않도록 기본적인 탐색
                temp_author_start = formatted_text.find(author, search_offset)
                if temp_author_start == -1:
                    break
                
                is_overlapping_with_title = False
                if title and title_start != -1:
                    title_range = range(title_start, title_start + len(title))
                    author_range = range(temp_author_start, temp_author_start + len(author))
                    if max(title_range.start, author_range.start) < min(title_range.stop, author_range.stop):
                        is_overlapping_with_title = True
                
                if not is_overlapping_with_title:
                    author_start = temp_author_start
                    break
                search_offset = temp_author_start + 1
            
            if author_start != -1:
                author_end = author_start + len(author)
                current_entities.append({
                    "start": author_start,
                    "end": author_end,
                    "type": author_entity_type
                })

        for keyword in format_keywords:
            start_pos = 0
            while True:
                keyword_start = formatted_text.find(keyword, start_pos)
                if keyword_start == -1:
                    break
                keyword_end = keyword_start + len(keyword)

                is_excluded = False
                for exclude_word in exclude_words:
                    # 키워드가 exclude_word의 일부인지 확인 (예: "도서" in "도서관")
                    # 좀 더 정확한 비교를 위해, exclude_word가 keyword_start 위치에서 시작하는지,
                    # 그리고 그 안에 keyword가 포함되는지 확인
                    if formatted_text.startswith(exclude_word, max(0, keyword_start - exclude_word.find(keyword) if keyword in exclude_word else 0)):
                         # keyword_start가 exclude_word 내의 keyword 위치와 일치하는지 확인
                        idx_in_exclude = exclude_word.find(keyword)
                        if idx_in_exclude != -1:
                            if keyword_start - idx_in_exclude >= 0 and \
                               formatted_text[keyword_start - idx_in_exclude : keyword_start - idx_in_exclude + len(exclude_word)] == exclude_word:
                                is_excluded = True
                                break
                if is_excluded:
                    start_pos = keyword_start + 1 
                    continue

                valid_endings = True
                if keyword_end < len(formatted_text):
                    next_char = formatted_text[keyword_end]
                    if '가' <= next_char <= '힣':
                        valid_suffixes = ['이', '가', '을', '를', '은', '는', '의', '에', '도', '만', '과', '와',
                                         '까지', '부터', '에서', '으로', '로', '들', '께', '랑', '이랑', '하고', '점']
                        found_valid_suffix = False
                        if not next_char.isalnum():
                            found_valid_suffix = True
                        else:
                            for suffix in valid_suffixes:
                                if formatted_text[keyword_end:].startswith(suffix):
                                    if len(formatted_text) == keyword_end + len(suffix) or \
                                       not formatted_text[keyword_end + len(suffix)].isalnum() or \
                                       not ('가' <= formatted_text[keyword_end + len(suffix)] <= '힣'):
                                        found_valid_suffix = True
                                        break
                        if not found_valid_suffix:
                            valid_endings = False
                
                valid_beginnings = True
                if keyword_start > 0:
                    prev_char = formatted_text[keyword_start-1]
                    if '가' <= prev_char <= '힣':
                        valid_beginnings = False

                if valid_endings and valid_beginnings:
                    is_overlapping_with_existing = False
                    new_entity_range = range(keyword_start, keyword_end)
                    for existing_entity in current_entities:
                        existing_entity_range = range(existing_entity["start"], existing_entity["end"])
                        if max(new_entity_range.start, existing_entity_range.start) < min(new_entity_range.stop, existing_entity_range.stop):
                            is_overlapping_with_existing = True
                            break
                    
                    if not is_overlapping_with_existing:
                        current_entities.append({
                            "start": keyword_start,
                            "end": keyword_end,
                            "type": "format"
                        })
                start_pos = keyword_end
        
        current_entities.sort(key=lambda x: x["start"])
        data_point = {"text": formatted_text, "intent": current_intent_label, "entities": current_entities}
        output_data.append(data_point)
    return output_data

# --- 파일 로드 ---
# 사용자의 원래 코드에서는 entityPath를 my_titles로 로드했으므로, 그 부분을 유지합니다.
# main_entity_type을 'BOOK'으로 설정하고 싶으면, entityMain 값을 'BOOK'으로 변경하세요.
# 현재는 전역변수 entityMain ('genre')과 entityAuthor ('author')를 사용합니다.
# 요청하신 형식 {"type": "BOOK"}에 맞추려면 스크립트 상단의 entityMain = 'genre'를
# entityMain = 'BOOK'으로 변경하거나, 아래 generate_ner_data 호출 시 직접 'BOOK'을 전달해야 합니다.
# 여기서는 후자의 방식을 택하여, entityMain 변수값과 관계없이 'BOOK' 타입을 사용하도록 수정합니다.

# 파일 로드 (오류 처리 포함)
try:
    with open(templatePath, 'r', encoding='utf-8') as f:
        my_templates = commentjson.load(f)
except FileNotFoundError:
    print(f"오류: 템플릿 파일({templatePath})을 찾을 수 없습니다.")
    my_templates = []

try:
    with open(entityPath, 'r', encoding='utf-8') as f:
        my_main_entities_from_file = commentjson.load(f) # 파일에서 읽은 엔티티 (원본의 my_titles에 해당)
except FileNotFoundError:
    print(f"오류: 주요 엔티티 파일({entityPath})을 찾을 수 없습니다.")
    my_main_entities_from_file = []

try:
    with open(authorPath, 'r', encoding='utf-8') as f:
        my_authors = commentjson.load(f)
except FileNotFoundError:
    print(f"정보: 작가 파일({authorPath})을 찾을 수 없습니다. 작가 정보 없이 진행합니다.")
    my_authors = []


# NER 데이터 생성
# entityMain의 값을 'BOOK'으로 직접 지정하여 요청하신 형식에 맞춤
# 만약 파일에 따라 동적으로 하고 싶다면 entityMain 변수를 사용
if my_templates and my_main_entities_from_file:
    generated_ner_data = generate_ner_data(
        my_templates,
        my_main_entities_from_file, # 파일에서 로드한 엔티티 전달
        my_authors,
        my_label,       # 인텐트 라벨
        "BOOK",         # 주요 엔티티 타입을 "BOOK"으로 고정 (요청사항 반영)
        entityAuthor    # 저자 엔티티 타입은 기존 변수 사용
    )
else:
    generated_ner_data = []
    if not my_templates: print("템플릿 데이터가 없어 NER 데이터를 생성할 수 없습니다.")
    if not my_main_entities_from_file: print("주요 엔티티 데이터가 없어 NER 데이터를 생성할 수 없습니다.")

# NER 데이터 출력
ner_output = json.dumps(generated_ner_data, ensure_ascii=False, indent=2)
print("\nNER 데이터:")
print(ner_output)


NER 데이터:
[
  {
    "text": "오힘찬의 플로라가 요즘 핫하던데.",
    "intent": 0,
    "entities": [
      {
        "start": 0,
        "end": 3,
        "type": "AUTHOR"
      },
      {
        "start": 5,
        "end": 8,
        "type": "BOOK"
      }
    ]
  },
  {
    "text": "스즈미야 하루히의 우울 있잖아 크리스토퍼 라이트이 쓴 책 그거 알아?",
    "intent": 0,
    "entities": [
      {
        "start": 0,
        "end": 12,
        "type": "BOOK"
      },
      {
        "start": 17,
        "end": 26,
        "type": "AUTHOR"
      }
    ]
  },
  {
    "text": "서승완작가의 신작 정관스님 나의 음식 읽어봤어?",
    "intent": 0,
    "entities": [
      {
        "start": 0,
        "end": 3,
        "type": "AUTHOR"
      },
      {
        "start": 10,
        "end": 20,
        "type": "BOOK"
      }
    ]
  },
  {
    "text": "요즘 베스트셀러라는 오기 마치 그거 이가라시 마사쿠니작가가 쓴 거 맞지?",
    "intent": 0,
    "entities": [
      {
        "start": 11,
        "end": 16,
        "type": "BOOK"
      },
      {
        "start": 20,
        "end": 29,
        "